<a href="https://colab.research.google.com/github/bhuvnk/END2.0/blob/main/Session%205%20-%20First%20Hands-on%20Sentiment%20Classification/Session5_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jun  3 21:49:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=2475174740f78aa1bfc3cf6b50e9efb357e806a156443b648c77802e68b1c782
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [3]:
import pytreebank
import sys
import os
import pandas as pd

In [4]:
# !wget http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
# !unzip -q 'stanfordSentimentTreebank.zip'

In [5]:
out_path = os.path.join(sys.path[0], 'sst_{}.txt')
dataset = pytreebank.load_sst('/content/stanfordSentimentTreebank')

for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))

In [6]:
train_df1 = pd.read_csv('./sst_train.txt',sep='\t',header=None,names=['label','text']) # reading the training set data

valid_df = pd.read_csv('./sst_dev.txt',sep='\t',header=None,names=['label','text']) # reading the validation set data
# train_df1 = pd.concat([train_df1, dev_df], ignore_index=True) # Concatinating dev with train

test_df = pd.read_csv('./sst_test.txt',sep='\t',header=None,names=['label','text']) # reading the test set data

In [7]:
train_df1.head()

,label,text
0,4,The Rock is destined to be the 21st Century 's...
1,5,The gorgeously elaborate continuation of `` Th...
2,4,Singer/composer Bryan Adams contributes a slew...
3,3,You 'd think by now America would have had eno...
4,4,Yet the act is still charming here .


In [11]:
train_df1.shape, valid_df.shape, test_df.shape

((8544, 2), (1101, 2), (2210, 2))

# Data Augmentation

### Back Translation

Seems to be one of the worse Augmentation when it comes to NLP.

In [13]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 1.0MB 17.1MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=4599b0cb6f94c3cc8bdeaeb26f9cf834f9b6d94ee03e9ab346a50edc3b12c5e7
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [14]:
import random
import googletrans
 
translator = googletrans.Translator()
available_langs = list(googletrans.LANGUAGES.keys()) 

In [15]:
vailable_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

Translating to sundanese


In [16]:
# Trying the backtranslation on random 200 sentences
sample = train_df1.sample(2000)
text = list(sample.text)

In [17]:
#translating to.
translations = translator.translate(text, dest=trans_lang) 
t_text = [t.text for t in translations]
# print(t_text)

#and back
translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
# print(en_text)

In [18]:
back_trans_df=pd.DataFrame({'text':en_text,'label':list(sample.label)})
back_trans_df.shape

(2000, 2)

In [19]:
back_trans_df.head()

,text,label
0,Ieu ngan ukur nyerat teu puguh.,1
1,"Éta pilem pilem baheula anu hadé, anu hartosna...",4
2,"Bogdanovich ngetok jero kana mistik Hearst, ng...",5
3,Anjeun tiasa ningali dimana Bad Bad Love nyobi...,3
4,... pasemon da'wah ditata ku sentuhan ballet p...,3


In [20]:
# train_df_agumented = pd.concat([train_df,d1])
# train_df_agumented = train_df_agumented.reset_index(drop=True)
# train_df_agumented.shape

### Radom swap

In [21]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [22]:
data_rand_swap=pd.DataFrame()
data_rand_swap['text']=train_df1['text'].apply(lambda x : " ".join(random_swap(x.split(' '))))
data_rand_swap['label'] = train_df1['label'] 

In [23]:
data_rand_swap.shape

(8544, 2)

In [24]:
# train_df_agumented1=pd.concat([train_df_agumented,data_rand_swap])
# train_df_agumented1=train_df_agumented1.reset_index(drop=True)
# train_df_agumented1.shape

### Random Deletion

In [25]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [26]:
data_rand_del=pd.DataFrame()
data_rand_del['text'] = train_df1['text'].apply(lambda x : " ".join(random_deletion(x.split(' '))))
data_rand_del['label'] = train_df1['label'] 

In [27]:
data_rand_del.shape

(8544, 2)

In [28]:
# train_df_agumented2=pd.concat([train_df_agumented1,data_rand_del])
# train_df_agumented2=train_df_agumented2.reset_index(drop=True)
# train_df_agumented2.shape

## combining augmented data with original data

In [29]:
train_df2 = pd.concat([train_df1, back_trans_df, data_rand_swap, data_rand_del], ignore_index=True)
train_df2.sample(10)

,label,text
11719,2,I version of think most n't people the who do ...
9124,2,You get the impression that writer and directo...
3094,5,"Not only is it a charming , funny and beautifu..."
5189,1,"A loud , low-budget and tired formula film tha..."
19205,5,Jeffrey Tambor performance intelligent jazz-pl...
15999,2,But the movie . muddled hook is effectively wa...
9520,4,I 'm not a fan of the phrase ` life affirming ...
12707,5,Spielberg 's deserved real masterpiece later s...
3396,4,Precocious smarter-than-thou wayward teen stru...
9583,3,Especially compared with the television series...


In [35]:
train_df2.shape

NameError: ignored

In [36]:
tran_df = train_df2.sample(train_df2.shape//2)

NameError: ignored

# Data Preparation for modeling

In [ ]:
import random
import torch, torchtext
from torchtext import legacy
from torchtext.legacy import data

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [37]:
Text = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [38]:
#Mapping dataframe fields to tensor fields 
fields=[('text',Text),('label',Label)]

Converting each sample into pyrorch example

In [39]:
example_train= [data.Example.fromlist([train_df.text[i],train_df.label[i]], fields) for i in range(train_df.shape[0])]

In [63]:
# Creating dataset
train = data.Dataset(example_train, fields)

In [41]:
example_valid=[data.Example.fromlist([valid_df.text[i],valid_df.label[i]],fields) for i in range (valid_df.shape[0])]

In [64]:
valid = data.Dataset(example_valid, fields)

In [43]:
example_test=[data.Example.fromlist([test_df.text[i],test_df.label[i]],fields) for i in range (test_df.shape[0])]

In [65]:
test = data.Dataset(example_test, fields)

In [66]:
vars(train.examples[10])

{'label': 5,
 'text': ['Good',
  'fun',
  ',',
  'good',
  'action',
  ',',
  'good',
  'acting',
  ',',
  'good',
  'dialogue',
  ',',
  'good',
  'pace',
  ',',
  'good',
  'cinematography',
  '.']}

In [67]:
vars(valid.examples[10])

{'label': 5,
 'text': ['Unlike',
  'the',
  'speedy',
  'wham',
  '-',
  'bam',
  'effect',
  'of',
  'most',
  'Hollywood',
  'offerings',
  ',',
  'character',
  'development',
  '--',
  'and',
  'more',
  'importantly',
  ',',
  'character',
  'empathy',
  '--',
  'is',
  'at',
  'the',
  'heart',
  'of',
  'Italian',
  'for',
  'Beginners',
  '.']}

Building the Vocabulary.

In [68]:
MAX_VOCAB_SIZE = 25_000

Text.build_vocab(train,
                 max_size = MAX_VOCAB_SIZE
                 ) #, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
Label.build_vocab(train)

In [69]:
print(f"Unique tokens in TEXT vocabulary: {len(Text.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(Label.vocab)}")

Unique tokens in TEXT vocabulary: 18686
Unique tokens in LABEL vocabulary: 5


Wanted some 25_000 vocab size but looks like we don't have enough words



In [70]:
print('Size of input vocab : ', len(Text.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Text.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18686
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 22012), (',', 19449), ('the', 16326), ('and', 12120), ('a', 11964), ('of', 11929), ('to', 8184), ('-', 7475), ("'s", 6839), ('is', 6802)]
Labels :  defaultdict(None, {4: 0, 2: 1, 3: 2, 5: 3, 1: 4})


A lot of stop words

In [71]:
BATCH_SIZE = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [72]:
train_iterator = data.BucketIterator(train, 
                                     batch_size = BATCH_SIZE,
                                     sort_key = lambda x: len(x.text),
                                     sort_within_batch=True, 
                                     device = device)
valid_iterator = data.BucketIterator(valid,
                                     batch_size = BATCH_SIZE, 
                                     sort_key = lambda x: len(x.text),
                                     sort_within_batch=True,
                                     device = device)
test_iterator = data.BucketIterator(test,
                                     batch_size = BATCH_SIZE, 
                                     sort_key = lambda x: len(x.text),
                                     sort_within_batch=True,
                                     device = device)

In [73]:
print('Train data dims')
for batch in train_iterator:
    print(f'input_text: {batch.text[0].size()}, length: {batch.text[1].size()}')
    print(f'Target: {batch.label.size()}')
    break

Train data dims
input_text: torch.Size([128, 26]), length: torch.Size([128])
Target: torch.Size([128])


In [74]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Text.vocab.stoi, tokens)

In [75]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                          #  bidirectional=True,
                           )
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [76]:
# Define hyperparameters
size_of_vocab = len(Text.vocab)
embedding_dim = 256
num_hidden_nodes = 256
num_output_nodes = 5
num_layers = 2
dropout = 0.5
PAD_IDX = Text.vocab.stoi[Text.pad_token]
# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout)

In [77]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(18686, 256)
  (encoder): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=5, bias=True)
)
The model has 5,837,573 trainable parameters


In [78]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [79]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        #tweet, tweet_lengths = batch.tweet  
        text,text_length = batch.text
        
        # convert to 1D tensor
        #predictions = model(tweet, tweet_lengths).squeeze()  
        predictions=model(text,text_length)
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [80]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            #tweet, tweet_lengths = batch.tweet
            text,text_length = batch.text
            
            # convert to 1d tensor
            predictions = model(text, text_length).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [81]:
import matplotlib.pyplot as plt
from IPython import display
plt.style.use('seaborn-white')
import numpy as np

In [82]:
N_EPOCHS = 30
best_valid_loss = float('inf')
epoch_list = np.zeros((0))
train_loss_list = np.zeros((0))
valid_loss_list = np.zeros((0))
for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights_lstm.pt')
    
    print(f'\t epoch : {epoch} |\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')
    epoch_list = np.append(epoch_list, [epoch])
    train_loss_list= np.append(train_loss_list, [train_loss])
    valid_loss_list= np.append(valid_loss_list, [valid_loss])

	 epoch : 0 |	Train Loss: 1.576 | Train Acc: 28.47%
	 Val. Loss: 1.583 |  Val. Acc: 27.60% 

	 epoch : 1 |	Train Loss: 1.540 | Train Acc: 34.77%
	 Val. Loss: 1.547 |  Val. Acc: 33.55% 

	 epoch : 2 |	Train Loss: 1.497 | Train Acc: 39.84%
	 Val. Loss: 1.540 |  Val. Acc: 34.31% 

	 epoch : 3 |	Train Loss: 1.454 | Train Acc: 44.51%
	 Val. Loss: 1.539 |  Val. Acc: 34.68% 

	 epoch : 4 |	Train Loss: 1.419 | Train Acc: 48.40%
	 Val. Loss: 1.549 |  Val. Acc: 34.45% 

	 epoch : 5 |	Train Loss: 1.385 | Train Acc: 51.98%
	 Val. Loss: 1.543 |  Val. Acc: 34.86% 

	 epoch : 6 |	Train Loss: 1.353 | Train Acc: 55.47%
	 Val. Loss: 1.553 |  Val. Acc: 34.42% 

	 epoch : 7 |	Train Loss: 1.324 | Train Acc: 58.42%
	 Val. Loss: 1.548 |  Val. Acc: 35.03% 

	 epoch : 8 |	Train Loss: 1.304 | Train Acc: 60.48%
	 Val. Loss: 1.556 |  Val. Acc: 31.96% 

	 epoch : 9 |	Train Loss: 1.267 | Train Acc: 64.55%
	 Val. Loss: 1.567 |  Val. Acc: 31.39% 

	 epoch : 10 |	Train Loss: 1.241 | Train Acc: 67.53%
	 Val. Loss: 1.55

## on test data

In [83]:
model.load_state_dict(torch.load('saved_weights_lstm.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.546 | Test Acc: 33.14%


## Few Predictions

In [88]:
#inference 
path='./saved_weights_lstm.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

import spacy
nlp = spacy.load('en')

def classify_sentences(sent):
        
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sent)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    #return categories[pred.item()]
    return pred.item()

In [94]:
for i in range(10):
  sent = ' '.join(test.examples[i].text)
  pred = classify_sentences(sent)
  print(f"Input: {sent}, \n \t label : {test.examples[i].label} \t predicted : {pred} \n")

Input: Effective but too - tepid biopic, 
 	 label : 3 	 predicted : 1 

Input: If you sometimes like to go to the movies to have fun , Wasabi is a good place to start ., 
 	 label : 4 	 predicted : 2 

Input: Emerges as something rare , an issue movie that 's so honest and keenly observed that it does n't feel like one ., 
 	 label : 5 	 predicted : 1 

Input: The film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game ., 
 	 label : 3 	 predicted : 0 

Input: Offers that rare combination of entertainment and education ., 
 	 label : 5 	 predicted : 0 

Input: Perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions ., 
 	 label : 4 	 predicted : 1 

Input: Steers turns in a snappy screenplay that curls at the edges ; it 's so clever you want to hate it ., 
 	 label : 4 	 predicted : 0 

Input: But he somehow pulls it off ., 
 	 label : 4 	 predicted : 1 

Inp